In [1]:
import csv
import hashlib
import time
import random
import string
import unittest
from datetime import datetime
from typing import List, Dict, Optional



def encrypt_password(password: str) -> str:
    """Encrypt password using SHA-256 hashing"""
    return hashlib.sha256(password.encode()).hexdigest()

def verify_password(stored_password: str, provided_password: str) -> bool:
    """Verify if provided password matches stored encrypted password"""
    return stored_password == encrypt_password(provided_password)

def generate_unique_id(prefix: str) -> str:
    """Generate unique ID with timestamp and random string"""
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    random_str = ''.join(random.choices(string.ascii_uppercase + string.digits, k=4))
    return f"{prefix}_{timestamp}_{random_str}"


# Grade Class


class Grade:
    """Represents grade information with ranges"""

    def __init__(self, grade_id: str, grade: str, min_marks: int, max_marks: int):
        self.grade_id = grade_id
        self.grade = grade
        self.min_marks = min_marks
        self.max_marks = max_marks

    def display_grade_info(self):
        """Display grade information"""
        print(f"Grade: {self.grade}, Range: {self.min_marks}-{self.max_marks}")

    @staticmethod
    def get_grade_from_marks(marks: float) -> str:
        """Convert marks to letter grade"""
        if marks >= 90:
            return 'A'
        elif marks >= 80:
            return 'B'
        elif marks >= 70:
            return 'C'
        elif marks >= 60:
            return 'D'
        else:
            return 'F'

    def to_dict(self) -> dict:
        """Convert Grade object to dictionary"""
        return {
            'grade_id': self.grade_id,
            'grade': self.grade,
            'min_marks': self.min_marks,
            'max_marks': self.max_marks
        }

# Course Class


class Course:
    """Represents course information"""

    def __init__(self, course_id: str, course_name: str, credits: int, description: str = ""):
        if not course_id:
            raise ValueError("Course ID cannot be empty")
        self.course_id = course_id
        self.course_name = course_name
        self.credits = credits
        self.description = description

    def display_course_details(self):
        """Display course information"""
        print(f"\nCourse ID: {self.course_id}")
        print(f"Course Name: {self.course_name}")
        print(f"Credits: {self.credits}")
        print(f"Description: {self.description}")

    def to_dict(self) -> dict:
        """Convert Course object to dictionary"""
        return {
            'course_id': self.course_id,
            'course_name': self.course_name,
            'credits': self.credits,
            'description': self.description
        }


# Professor Class


class Professor:
    """Represents professor information - HAS-A relationship with Course"""

    def __init__(self, professor_id: str, name: str, email: str, rank: str, course_ids: List[str] = None):
        if not professor_id or not email:
            raise ValueError("Professor ID and Email cannot be empty")
        self.professor_id = professor_id
        self.name = name
        self.email = email
        self.rank = rank
        self.course_ids = course_ids if course_ids else []

    def display_professor_details(self):
        """Display professor information"""
        print(f"\nProfessor ID: {self.professor_id}")
        print(f"Name: {self.name}")
        print(f"Email: {self.email}")
        print(f"Rank: {self.rank}")
        print(f"Courses: {', '.join(self.course_ids)}")

    def add_course(self, course_id: str):
        """Add course to professor's teaching list"""
        if course_id not in self.course_ids:
            self.course_ids.append(course_id)

    def remove_course(self, course_id: str):
        """Remove course from professor's teaching list"""
        if course_id in self.course_ids:
            self.course_ids.remove(course_id)

    def to_dict(self) -> dict:
        """Convert Professor object to dictionary"""
        return {
            'professor_id': self.professor_id,
            'name': self.name,
            'email': self.email,
            'rank': self.rank,
            'course_ids': ','.join(self.course_ids)
        }


# Student Class


class Student:
    """Represents student information with grades - HAS-A relationship with Course and Grade"""

    def __init__(self, student_id: str, first_name: str, last_name: str, email: str):
        if not student_id or not email:
            raise ValueError("Student ID and Email cannot be empty")
        self.student_id = student_id
        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.courses = {}  # {course_id: {'marks': float, 'grade': str}}

    def display_student_info(self):
        """Display student information"""
        print(f"\nStudent ID: {self.student_id}")
        print(f"Name: {self.first_name} {self.last_name}")
        print(f"Email: {self.email}")
        if self.courses:
            print("Enrolled Courses:")
            for course_id, data in self.courses.items():
                print(f"  {course_id}: Marks={data['marks']}, Grade={data['grade']}")
        else:
            print("No courses enrolled")

    def add_course_grade(self, course_id: str, marks: float):
        """Add or update course grade for student"""
        grade = Grade.get_grade_from_marks(marks)
        self.courses[course_id] = {'marks': marks, 'grade': grade}

    def remove_course(self, course_id: str):
        """Remove course from student's record"""
        if course_id in self.courses:
            del self.courses[course_id]

    def get_average_marks(self) -> float:
        """Calculate average marks across all courses"""
        if not self.courses:
            return 0.0
        total = sum(data['marks'] for data in self.courses.values())
        return total / len(self.courses)

    def to_dict(self) -> dict:
        """Convert Student object to dictionary for CSV"""
        course_ids = ','.join(self.courses.keys())
        grades = ','.join(data['grade'] for data in self.courses.values())
        marks = ','.join(str(data['marks']) for data in self.courses.values())

        return {
            'student_id': self.student_id,
            'first_name': self.first_name,
            'last_name': self.last_name,
            'email': self.email,
            'course_ids': course_ids,
            'grades': grades,
            'marks': marks
        }


# LoginUser Class


class LoginUser:
    """Handles user authentication - IS-A relationship (can be Student, Professor, or Admin)"""

    def __init__(self, email: str, password: str, role: str):
        self.email = email
        self.password_hash = encrypt_password(password)
        self.role = role
        self.is_logged_in = False

    def login(self, provided_password: str) -> bool:
        """Authenticate user with password"""
        if verify_password(self.password_hash, provided_password):
            self.is_logged_in = True
            print(f"Login successful! Welcome {self.email}")
            return True
        else:
            print("Invalid password")
            return False

    def logout(self):
        """Logout user"""
        self.is_logged_in = False
        print("Logged out successfully")

    def change_password(self, old_password: str, new_password: str) -> bool:
        """Change user password"""
        if verify_password(self.password_hash, old_password):
            self.password_hash = encrypt_password(new_password)
            print("Password changed successfully")
            return True
        else:
            print("Old password incorrect")
            return False

    def to_dict(self) -> dict:
        """Convert LoginUser to dictionary"""
        return {
            'email': self.email,
            'password': self.password_hash,
            'role': self.role
        }

# CheckMyGrade Application - Main Class

class CheckMyGradeApp:
    """Main application class managing all entities"""

    def __init__(self):
        self.students = []
        self.courses = []
        self.professors = []
        self.users = []
        self.grades = self._initialize_grades()
        self.current_user = None

    def _initialize_grades(self) -> List[Grade]:
        """Initialize standard grading system"""
        return [
            Grade("G1", "A", 90, 100),
            Grade("G2", "B", 80, 89),
            Grade("G3", "C", 70, 79),
            Grade("G4", "D", 60, 69),
            Grade("G5", "F", 0, 59)
        ]

    # ==================== Student Operations ====================

    def add_student(self, first_name: str, last_name: str, email: str) -> Student:
        """Add new student to the system"""
        if any(s.email == email for s in self.students):
            raise ValueError(f"Student with email {email} already exists")

        student_id = generate_unique_id("STU")
        student = Student(student_id, first_name, last_name, email)
        self.students.append(student)
        print(f"Student {first_name} {last_name} added successfully with ID: {student_id}")
        return student

    def delete_student(self, student_id: str) -> bool:
        """Delete student by ID"""
        for i, student in enumerate(self.students):
            if student.student_id == student_id:
                self.students.pop(i)
                print(f"Student {student_id} deleted successfully")
                return True
        print(f"Student {student_id} not found")
        return False

    def update_student(self, student_id: str, **kwargs) -> bool:
        """Update student information"""
        student = self.search_student_by_id(student_id)
        if student:
            if 'first_name' in kwargs:
                student.first_name = kwargs['first_name']
            if 'last_name' in kwargs:
                student.last_name = kwargs['last_name']
            if 'email' in kwargs:
                student.email = kwargs['email']
            print(f"Student {student_id} updated successfully")
            return True
        print(f"Student {student_id} not found")
        return False

    def search_student_by_id(self, student_id: str) -> Optional[Student]:
        """Search student by ID - Linear Search"""
        for student in self.students:
            if student.student_id == student_id:
                return student
        return None

    def search_student_by_email(self, email: str) -> Optional[Student]:
        """Search student by email"""
        for student in self.students:
            if student.email == email:
                return student
        return None

    def sort_students_by_name(self, reverse: bool = False):
        """Sort students by name using bubble sort"""
        n = len(self.students)
        for i in range(n):
            for j in range(0, n - i - 1):
                name1 = f"{self.students[j].first_name} {self.students[j].last_name}"
                name2 = f"{self.students[j + 1].first_name} {self.students[j + 1].last_name}"
                if (name1 > name2 and not reverse) or (name1 < name2 and reverse):
                    self.students[j], self.students[j + 1] = self.students[j + 1], self.students[j]

    def sort_students_by_marks(self, reverse: bool = False):
        """Sort students by average marks using selection sort"""
        n = len(self.students)
        for i in range(n):
            target_idx = i
            for j in range(i + 1, n):
                avg1 = self.students[target_idx].get_average_marks()
                avg2 = self.students[j].get_average_marks()
                if (avg2 > avg1 and not reverse) or (avg2 < avg1 and reverse):
                    target_idx = j
            self.students[i], self.students[target_idx] = self.students[target_idx], self.students[i]

    # ==================== Course Operations ====================

    def add_course(self, course_id: str, course_name: str, credits: int, description: str = "") -> Course:
        """Add new course"""
        if any(c.course_id == course_id for c in self.courses):
            raise ValueError(f"Course with ID {course_id} already exists")

        course = Course(course_id, course_name, credits, description)
        self.courses.append(course)
        print(f"Course {course_name} added successfully")
        return course

    def delete_course(self, course_id: str) -> bool:
        """Delete course by ID"""
        for i, course in enumerate(self.courses):
            if course.course_id == course_id:
                self.courses.pop(i)
                print(f"Course {course_id} deleted successfully")
                return True
        print(f"Course {course_id} not found")
        return False

    def update_course(self, course_id: str, **kwargs) -> bool:
        """Update course information"""
        course = self.search_course_by_id(course_id)
        if course:
            if 'course_name' in kwargs:
                course.course_name = kwargs['course_name']
            if 'credits' in kwargs:
                course.credits = kwargs['credits']
            if 'description' in kwargs:
                course.description = kwargs['description']
            print(f"Course {course_id} updated successfully")
            return True
        print(f"Course {course_id} not found")
        return False

    def search_course_by_id(self, course_id: str) -> Optional[Course]:
        """Search course by ID"""
        for course in self.courses:
            if course.course_id == course_id:
                return course
        return None

    # ==================== Professor Operations ====================

    def add_professor(self, name: str, email: str, rank: str, course_ids: List[str] = None) -> Professor:
        """Add new professor"""
        if any(p.email == email for p in self.professors):
            raise ValueError(f"Professor with email {email} already exists")

        professor_id = generate_unique_id("PROF")
        professor = Professor(professor_id, name, email, rank, course_ids)
        self.professors.append(professor)
        print(f"Professor {name} added successfully with ID: {professor_id}")
        return professor

    def delete_professor(self, professor_id: str) -> bool:
        """Delete professor by ID"""
        for i, prof in enumerate(self.professors):
            if prof.professor_id == professor_id:
                self.professors.pop(i)
                print(f"Professor {professor_id} deleted successfully")
                return True
        print(f"Professor {professor_id} not found")
        return False

    def update_professor(self, professor_id: str, **kwargs) -> bool:
        """Update professor information"""
        professor = self.search_professor_by_id(professor_id)
        if professor:
            if 'name' in kwargs:
                professor.name = kwargs['name']
            if 'email' in kwargs:
                professor.email = kwargs['email']
            if 'rank' in kwargs:
                professor.rank = kwargs['rank']
            print(f"Professor {professor_id} updated successfully")
            return True
        print(f"Professor {professor_id} not found")
        return False

    def search_professor_by_id(self, professor_id: str) -> Optional[Professor]:
        """Search professor by ID"""
        for professor in self.professors:
            if professor.professor_id == professor_id:
                return professor
        return None

    # ==================== Grade Operations ====================

    def add_student_grade(self, student_id: str, course_id: str, marks: float):
        """Add grade for a student in a course"""
        student = self.search_student_by_id(student_id)
        if not student:
            print(f"Student {student_id} not found")
            return False

        course = self.search_course_by_id(course_id)
        if not course:
            print(f"Course {course_id} not found")
            return False

        student.add_course_grade(course_id, marks)
        print(f"Grade added for student {student_id} in course {course_id}")
        return True

    # ==================== Statistics Operations ====================

    def get_course_statistics(self, course_id: str) -> dict:
        """Calculate statistics for a course"""
        marks_list = []
        for student in self.students:
            if course_id in student.courses:
                marks_list.append(student.courses[course_id]['marks'])

        if not marks_list:
            return {'average': 0, 'median': 0, 'count': 0}

        average = sum(marks_list) / len(marks_list)

        sorted_marks = sorted(marks_list)
        n = len(sorted_marks)
        if n % 2 == 0:
            median = (sorted_marks[n//2 - 1] + sorted_marks[n//2]) / 2
        else:
            median = sorted_marks[n//2]

        return {
            'average': round(average, 2),
            'median': round(median, 2),
            'count': len(marks_list),
            'min': min(marks_list),
            'max': max(marks_list)
        }

    # ==================== Report Operations ====================

    def display_student_report(self, student_id: str):
        """Display complete report for a student"""
        student = self.search_student_by_id(student_id)
        if student:
            student.display_student_info()
            avg = student.get_average_marks()
            print(f"Average Marks: {avg:.2f}")
        else:
            print(f"Student {student_id} not found")

    def display_course_report(self, course_id: str):
        """Display report for a course"""
        course = self.search_course_by_id(course_id)
        if not course:
            print(f"Course {course_id} not found")
            return

        course.display_course_details()
        stats = self.get_course_statistics(course_id)
        print(f"\nCourse Statistics:")
        print(f"  Students Enrolled: {stats['count']}")
        print(f"  Average Marks: {stats['average']}")
        print(f"  Median Marks: {stats['median']}")
        if stats['count'] > 0:
            print(f"  Highest Marks: {stats['max']}")
            print(f"  Lowest Marks: {stats['min']}")

    def display_professor_courses(self, professor_id: str):
        """Display all courses taught by a professor"""
        professor = self.search_professor_by_id(professor_id)
        if not professor:
            print(f"Professor {professor_id} not found")
            return

        professor.display_professor_details()
        print("\nCourses Taught:")
        for course_id in professor.course_ids:
            course = self.search_course_by_id(course_id)
            if course:
                course.display_course_details()

    # ==================== CSV File Operations ====================

    def save_students_to_csv(self, filename: str = "students.csv"):
        """Save all students to CSV file"""
        if not self.students:
            print("No students to save")
            return

        with open(filename, 'w', newline='') as file:
            fieldnames = ['student_id', 'first_name', 'last_name', 'email', 'course_ids', 'grades', 'marks']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for student in self.students:
                writer.writerow(student.to_dict())
        print(f"Students saved to {filename}")

    def load_students_from_csv(self, filename: str = "students.csv"):
        """Load students from CSV file"""
        try:
            with open(filename, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    student = Student(row['student_id'], row['first_name'],
                                    row['last_name'], row['email'])

                    if row['course_ids']:
                        course_ids = row['course_ids'].split(',')
                        grades = row['grades'].split(',')
                        marks_list = row['marks'].split(',')

                        for cid, grade, mark in zip(course_ids, grades, marks_list):
                            student.add_course_grade(cid, float(mark))

                    self.students.append(student)
            print(f"Students loaded from {filename}")
        except FileNotFoundError:
            print(f"File {filename} not found")

    def save_courses_to_csv(self, filename: str = "courses.csv"):
        """Save all courses to CSV file"""
        if not self.courses:
            print("No courses to save")
            return

        with open(filename, 'w', newline='') as file:
            fieldnames = ['course_id', 'course_name', 'credits', 'description']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for course in self.courses:
                writer.writerow(course.to_dict())
        print(f"Courses saved to {filename}")

    def load_courses_from_csv(self, filename: str = "courses.csv"):
        """Load courses from CSV file"""
        try:
            with open(filename, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    course = Course(row['course_id'], row['course_name'],
                                  int(row['credits']), row['description'])
                    self.courses.append(course)
            print(f"Courses loaded from {filename}")
        except FileNotFoundError:
            print(f"File {filename} not found")

    def save_professors_to_csv(self, filename: str = "professors.csv"):
        """Save all professors to CSV file"""
        if not self.professors:
            print("No professors to save")
            return

        with open(filename, 'w', newline='') as file:
            fieldnames = ['professor_id', 'name', 'email', 'rank', 'course_ids']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for professor in self.professors:
                writer.writerow(professor.to_dict())
        print(f"Professors saved to {filename}")

    def load_professors_from_csv(self, filename: str = "professors.csv"):
        """Load professors from CSV file"""
        try:
            with open(filename, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    course_ids = row['course_ids'].split(',') if row['course_ids'] else []
                    professor = Professor(row['professor_id'], row['name'],
                                        row['email'], row['rank'], course_ids)
                    self.professors.append(professor)
            print(f"Professors loaded from {filename}")
        except FileNotFoundError:
            print(f"File {filename} not found")

    def save_users_to_csv(self, filename: str = "login.csv"):
        """Save all users to CSV file"""
        if not self.users:
            print("No users to save")
            return

        with open(filename, 'w', newline='') as file:
            fieldnames = ['email', 'password', 'role']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for user in self.users:
                writer.writerow(user.to_dict())
        print(f"Users saved to {filename}")

    def load_users_from_csv(self, filename: str = "login.csv"):
        """Load users from CSV file"""
        try:
            with open(filename, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    user = LoginUser.__new__(LoginUser)
                    user.email = row['email']
                    user.password_hash = row['password']
                    user.role = row['role']
                    user.is_logged_in = False
                    self.users.append(user)
            print(f"Users loaded from {filename}")
        except FileNotFoundError:
            print(f"File {filename} not found")

    def save_all_data(self):
        """Save all data to CSV files"""
        self.save_students_to_csv()
        self.save_courses_to_csv()
        self.save_professors_to_csv()
        self.save_users_to_csv()
        print("\nAll data saved successfully!")

    def load_all_data(self):
        """Load all data from CSV files"""
        self.load_students_from_csv()
        self.load_courses_from_csv()
        self.load_professors_from_csv()
        self.load_users_from_csv()
        print("\nAll data loaded successfully!")


# Unit Tests


class TestCheckMyGradeApp(unittest.TestCase):
    """Comprehensive unit tests for CheckMyGrade Application"""

    def setUp(self):
        """Set up test fixtures before each test"""
        self.app = CheckMyGradeApp()

    def test_01_add_student(self):
        """Test adding students to the system"""
        print("\n" + "="*60)
        print("TEST 1: Adding Students")
        print("="*60)

        start_time = time.time()
        student = self.app.add_student("John", "Doe", "john@student.edu")
        end_time = time.time()

        self.assertIsNotNone(student)
        self.assertEqual(student.first_name, "John")
        self.assertEqual(len(self.app.students), 1)

        print(f"✓ Student added successfully")
        print(f"Execution time: {(end_time - start_time)*1000:.4f} milliseconds")

    def test_02_delete_student(self):
        """Test deleting students"""
        print("\n" + "="*60)
        print("TEST 2: Deleting Students")
        print("="*60)

        student = self.app.add_student("Jane", "Smith", "jane@student.edu")
        student_id = student.student_id

        start_time = time.time()
        result = self.app.delete_student(student_id)
        end_time = time.time()

        self.assertTrue(result)
        self.assertEqual(len(self.app.students), 0)

        print(f"✓ Student deleted successfully")
        print(f"Execution time: {(end_time - start_time)*1000:.4f} milliseconds")

    def test_03_update_student(self):
        """Test updating student information"""
        print("\n" + "="*60)
        print("TEST 3: Updating Student Records")
        print("="*60)

        student = self.app.add_student("Bob", "Johnson", "bob@student.edu")

        start_time = time.time()
        result = self.app.update_student(student.student_id, first_name="Robert")
        end_time = time.time()

        self.assertTrue(result)
        self.assertEqual(student.first_name, "Robert")

        print(f"✓ Student updated successfully")
        print(f"Execution time: {(end_time - start_time)*1000:.4f} milliseconds")

    def test_04_add_large_student_dataset(self):
        """Test adding 1000+ student records"""
        print("\n" + "="*60)
        print("TEST 4: Adding 1000+ Student Records")
        print("="*60)

        start_time = time.time()
        for i in range(1000):
            self.app.add_student(f"Student{i}", f"Last{i}", f"student{i}@edu.com")
        end_time = time.time()

        self.assertEqual(len(self.app.students), 1000)

        print(f"✓ 1000 students added successfully")
        print(f"Total execution time: {(end_time - start_time)*1000:.2f} milliseconds")
        print(f"Average time per student: {((end_time - start_time)*1000)/1000:.4f} milliseconds")

    def test_05_search_performance(self):
        """Test search performance with large dataset"""
        print("\n" + "="*60)
        print("TEST 5: Search Performance Test")
        print("="*60)

        # Add 1000 students
        for i in range(1000):
            self.app.add_student(f"Student{i}", f"Last{i}", f"student{i}@edu.com")

        # Search for student in beginning
        start_time = time.time()
        result = self.app.search_student_by_email("student5@edu.com")
        end_time = time.time()
        search_time_start = (end_time - start_time) * 1000

        self.assertIsNotNone(result)
        print(f"✓ Search at beginning: {search_time_start:.4f} milliseconds")

        # Search for student in middle
        start_time = time.time()
        result = self.app.search_student_by_email("student500@edu.com")
        end_time = time.time()
        search_time_mid = (end_time - start_time) * 1000

        self.assertIsNotNone(result)
        print(f"✓ Search in middle: {search_time_mid:.4f} milliseconds")

        # Search for student at end
        start_time = time.time()
        result = self.app.search_student_by_email("student999@edu.com")
        end_time = time.time()
        search_time_end = (end_time - start_time) * 1000

        self.assertIsNotNone(result)
        print(f"✓ Search at end: {search_time_end:.4f} milliseconds")
        print(f"Average search time: {(search_time_start + search_time_mid + search_time_end)/3:.4f} milliseconds")

    def test_06_sort_by_name(self):
        """Test sorting students by name"""
        print("\n" + "="*60)
        print("TEST 6: Sort Students by Name")
        print("="*60)

        # Add students
        self.app.add_student("Zara", "Adams", "zara@edu.com")
        self.app.add_student("Alice", "Brown", "alice@edu.com")
        self.app.add_student("Mike", "Chen", "mike@edu.com")

        start_time = time.time()
        self.app.sort_students_by_name()
        end_time = time.time()

        # Check if sorted correctly (ascending)
        self.assertEqual(self.app.students[0].first_name, "Alice")
        self.assertEqual(self.app.students[1].first_name, "Mike")
        self.assertEqual(self.app.students[2].first_name, "Zara")

        print(f"✓ Students sorted by name (ascending)")
        print(f"Execution time: {(end_time - start_time)*1000:.4f} milliseconds")

        # Test descending sort
        start_time = time.time()
        self.app.sort_students_by_name(reverse=True)
        end_time = time.time()

        self.assertEqual(self.app.students[0].first_name, "Zara")
        print(f"✓ Students sorted by name (descending)")
        print(f"Execution time: {(end_time - start_time)*1000:.4f} milliseconds")

    def test_07_sort_by_marks(self):
        """Test sorting students by marks"""
        print("\n" + "="*60)
        print("TEST 7: Sort Students by Marks")
        print("="*60)

        # Add students with grades
        s1 = self.app.add_student("Alice", "Cooper", "alice@edu.com")
        s2 = self.app.add_student("Bob", "Martin", "bob@edu.com")
        s3 = self.app.add_student("Charlie", "Brown", "charlie@edu.com")

        # Add course first
        self.app.add_course("DATA200", "Data Structures", 3, "DS Course")

        # Add grades
        self.app.add_student_grade(s1.student_id, "DATA200", 85)
        self.app.add_student_grade(s2.student_id, "DATA200", 95)
        self.app.add_student_grade(s3.student_id, "DATA200", 75)

        start_time = time.time()
        self.app.sort_students_by_marks(reverse=True)
        end_time = time.time()

        # Check if sorted correctly (highest to lowest)
        self.assertEqual(self.app.students[0].first_name, "Bob")
        self.assertEqual(self.app.students[1].first_name, "Alice")
        self.assertEqual(self.app.students[2].first_name, "Charlie")

        print(f"✓ Students sorted by marks (highest to lowest)")
        print(f"Execution time: {(end_time - start_time)*1000:.4f} milliseconds")

    def test_08_sort_large_dataset(self):
        """Test sorting performance with 1000+ records"""
        print("\n" + "="*60)
        print("TEST 8: Sort 1000+ Records Performance")
        print("="*60)

        # Add 1000 students
        for i in range(1000):
            self.app.add_student(f"Student{i}", f"Last{999-i}", f"student{i}@edu.com")

        # Sort by name
        start_time = time.time()
        self.app.sort_students_by_name()
        end_time = time.time()

        print(f"✓ 1000 students sorted by name")
        print(f"Execution time: {(end_time - start_time)*1000:.2f} milliseconds")

        # Add grades for sorting by marks
        self.app.add_course("DATA200", "Data Structures", 3)
        for student in self.app.students[:100]:  # Add grades to first 100
            marks = random.uniform(60, 100)
            self.app.add_student_grade(student.student_id, "DATA200", marks)

        start_time = time.time()
        self.app.sort_students_by_marks(reverse=True)
        end_time = time.time()

        print(f"✓ 1000 students sorted by marks")
        print(f"Execution time: {(end_time - start_time)*1000:.2f} milliseconds")

    def test_09_course_operations(self):
        """Test course add/delete/modify operations"""
        print("\n" + "="*60)
        print("TEST 9: Course Operations (Add/Delete/Modify)")
        print("="*60)

        # Test Add
        start_time = time.time()
        course = self.app.add_course("DATA201", "Machine Learning", 3, "ML Course")
        add_time = (time.time() - start_time) * 1000

        self.assertIsNotNone(course)
        self.assertEqual(len(self.app.courses), 1)
        print(f"✓ Course added - Time: {add_time:.4f} ms")

        # Test Modify
        start_time = time.time()
        result = self.app.update_course("DATA201", course_name="Advanced Machine Learning")
        modify_time = (time.time() - start_time) * 1000

        self.assertTrue(result)
        self.assertEqual(course.course_name, "Advanced Machine Learning")
        print(f"✓ Course modified - Time: {modify_time:.4f} ms")

        # Test Delete
        start_time = time.time()
        result = self.app.delete_course("DATA201")
        delete_time = (time.time() - start_time) * 1000

        self.assertTrue(result)
        self.assertEqual(len(self.app.courses), 0)
        print(f"✓ Course deleted - Time: {delete_time:.4f} ms")

    def test_10_professor_operations(self):
        """Test professor add/delete/modify operations"""
        print("\n" + "="*60)
        print("TEST 10: Professor Operations (Add/Delete/Modify)")
        print("="*60)

        # Add course first
        self.app.add_course("DATA200", "Data Structures", 3)

        # Test Add
        start_time = time.time()
        prof = self.app.add_professor("Dr. John Smith", "john@mycsu.edu",
                                      "Senior Professor", ["DATA200"])
        add_time = (time.time() - start_time) * 1000

        self.assertIsNotNone(prof)
        self.assertEqual(len(self.app.professors), 1)
        print(f"✓ Professor added - Time: {add_time:.4f} ms")

        # Test Modify
        start_time = time.time()
        result = self.app.update_professor(prof.professor_id, rank="Distinguished Professor")
        modify_time = (time.time() - start_time) * 1000

        self.assertTrue(result)
        self.assertEqual(prof.rank, "Distinguished Professor")
        print(f"✓ Professor modified - Time: {modify_time:.4f} ms")

        # Test Delete
        start_time = time.time()
        result = self.app.delete_professor(prof.professor_id)
        delete_time = (time.time() - start_time) * 1000

        self.assertTrue(result)
        self.assertEqual(len(self.app.professors), 0)
        print(f"✓ Professor deleted - Time: {delete_time:.4f} ms")

    def test_11_file_operations(self):
        """Test CSV file save and load operations"""
        print("\n" + "="*60)
        print("TEST 11: CSV File Operations (Save/Load)")
        print("="*60)

        # Add sample data
        self.app.add_course("DATA200", "Data Structures", 3, "DS Course")
        student = self.app.add_student("Test", "Student", "test@edu.com")
        self.app.add_professor("Dr. Test", "test@prof.edu", "Professor", ["DATA200"])
        self.app.add_student_grade(student.student_id, "DATA200", 90)

        # Test Save
        start_time = time.time()
        self.app.save_all_data()
        save_time = (time.time() - start_time) * 1000

        print(f"✓ All data saved - Time: {save_time:.4f} ms")

        # Create new app and load
        new_app = CheckMyGradeApp()
        start_time = time.time()
        new_app.load_all_data()
        load_time = (time.time() - start_time) * 1000

        self.assertEqual(len(new_app.students), 1)
        self.assertEqual(len(new_app.courses), 1)
        self.assertEqual(len(new_app.professors), 1)

        print(f"✓ All data loaded - Time: {load_time:.4f} ms")

    def test_12_statistics(self):
        """Test course statistics calculation"""
        print("\n" + "="*60)
        print("TEST 12: Course Statistics (Average/Median)")
        print("="*60)

        # Setup course and students
        self.app.add_course("DATA200", "Data Structures", 3)
        s1 = self.app.add_student("Student1", "Last1", "s1@edu.com")
        s2 = self.app.add_student("Student2", "Last2", "s2@edu.com")
        s3 = self.app.add_student("Student3", "Last3", "s3@edu.com")

        # Add grades
        self.app.add_student_grade(s1.student_id, "DATA200", 80)
        self.app.add_student_grade(s2.student_id, "DATA200", 90)
        self.app.add_student_grade(s3.student_id, "DATA200", 70)

        start_time = time.time()
        stats = self.app.get_course_statistics("DATA200")
        calc_time = (time.time() - start_time) * 1000

        self.assertEqual(stats['average'], 80.0)
        self.assertEqual(stats['median'], 80.0)
        self.assertEqual(stats['count'], 3)

        print(f"✓ Statistics calculated")
        print(f"  Average: {stats['average']}")
        print(f"  Median: {stats['median']}")
        print(f"  Count: {stats['count']}")
        print(f"Execution time: {calc_time:.4f} ms")

    def test_13_password_encryption(self):
        """Test password encryption and verification"""
        print("\n" + "="*60)
        print("TEST 13: Password Encryption and Login")
        print("="*60)

        # Create user
        start_time = time.time()
        user = LoginUser("test@user.com", "Welcome12#_", "student")
        encrypt_time = (time.time() - start_time) * 1000

        print(f"✓ Password encrypted - Time: {encrypt_time:.4f} ms")
        print(f"Original password: Welcome12#_")
        print(f"Encrypted hash: {user.password_hash[:32]}...")

        # Test correct login
        start_time = time.time()
        result = user.login("Welcome12#_")
        login_time = (time.time() - start_time) * 1000

        self.assertTrue(result)
        print(f"✓ Login successful - Time: {login_time:.4f} ms")

        # Test incorrect login
        user2 = LoginUser("test2@user.com", "Password123", "student")
        result = user2.login("WrongPassword")
        self.assertFalse(result)
        print(f"✓ Invalid login rejected correctly")


# Complete Demo with All Features


def run_complete_demo():
    """Run comprehensive demonstration of all features"""
    print("\n")
    print("="*70)
    print(" "*15 + "CheckMyGrade Application")
    print(" "*15 + "Complete Feature Demonstration")
    print("="*70)

    app = CheckMyGradeApp()

    # ========== SECTION 1: Course Management ==========
    print("\n" + "="*70)
    print("SECTION 1: COURSE MANAGEMENT")
    print("="*70)

    print("\n--- Adding Courses ---")
    app.add_course("DATA200", "Data Structures and Algorithms", 3,
                   "Learn fundamental data structures and algorithms")
    app.add_course("DATA201", "Machine Learning Fundamentals", 3,
                   "Introduction to ML concepts and techniques")
    app.add_course("DATA202", "Database Systems", 3,
                   "Relational and NoSQL database design")
    app.add_course("DATA203", "Big Data Analytics", 3,
                   "Processing and analyzing large datasets")

    print(f"\nTotal courses added: {len(app.courses)}")

    # ========== SECTION 2: Professor Management ==========
    print("\n" + "="*70)
    print("SECTION 2: PROFESSOR MANAGEMENT")
    print("="*70)

    print("\n--- Adding Professors ---")
    prof1 = app.add_professor("Dr. Michael Johnson", "michael@mycsu.edu",
                              "Senior Professor", ["DATA200", "DATA201"])
    prof2 = app.add_professor("Dr. Sarah Williams", "sarah@mycsu.edu",
                              "Associate Professor", ["DATA202"])
    prof3 = app.add_professor("Dr. Robert Chen", "robert@mycsu.edu",
                              "Assistant Professor", ["DATA203"])

    print(f"\nTotal professors added: {len(app.professors)}")

    # Display professor details
    print("\n--- Professor Course Details ---")
    app.display_professor_courses(prof1.professor_id)

    # ========== SECTION 3: Student Management ==========
    print("\n" + "="*70)
    print("SECTION 3: STUDENT MANAGEMENT")
    print("="*70)

    print("\n--- Adding Students ---")
    students_data = [
        ("Alice", "Cooper", "alice.cooper@student.edu"),
        ("Bob", "Martin", "bob.martin@student.edu"),
        ("Charlie", "Brown", "charlie.brown@student.edu"),
        ("Diana", "Prince", "diana.prince@student.edu"),
        ("Edward", "Norton", "edward.norton@student.edu"),
        ("Fiona", "Apple", "fiona.apple@student.edu"),
        ("George", "Miller", "george.miller@student.edu"),
        ("Hannah", "Montana", "hannah.montana@student.edu"),
    ]

    added_students = []
    for first, last, email in students_data:
        student = app.add_student(first, last, email)
        added_students.append(student)

    print(f"\nTotal students added: {len(app.students)}")

    # ========== SECTION 4: Grade Management ==========
    print("\n" + "="*70)
    print("SECTION 4: GRADE MANAGEMENT")
    print("="*70)

    print("\n--- Assigning Grades to Students ---")
    # Assign grades for DATA200
    grades_data = [
        (added_students[0].student_id, "DATA200", 95),
        (added_students[1].student_id, "DATA200", 78),
        (added_students[2].student_id, "DATA200", 85),
        (added_students[3].student_id, "DATA200", 92),
        (added_students[4].student_id, "DATA200", 88),
        (added_students[5].student_id, "DATA200", 76),
        (added_students[6].student_id, "DATA200", 90),
        (added_students[7].student_id, "DATA200", 83),
    ]

    for sid, cid, marks in grades_data:
        app.add_student_grade(sid, cid, marks)

    # Assign grades for DATA201
    for i, student in enumerate(added_students[:6]):
        marks = random.randint(70, 100)
        app.add_student_grade(student.student_id, "DATA201", marks)

    print(f"✓ Grades assigned successfully")

    # ========== SECTION 5: Reports ==========
    print("\n" + "="*70)
    print("SECTION 5: STUDENT AND COURSE REPORTS")
    print("="*70)

    print("\n--- Individual Student Report ---")
    app.display_student_report(added_students[0].student_id)

    print("\n--- Course Statistics Report ---")
    app.display_course_report("DATA200")

    # ========== SECTION 6: Search Operations ==========
    print("\n" + "="*70)
    print("SECTION 6: SEARCH OPERATIONS WITH TIMING")
    print("="*70)

    print("\n--- Searching Student by Email ---")
    start_time = time.time()
    result = app.search_student_by_email("alice.cooper@student.edu")
    search_time = (time.time() - start_time) * 1000

    if result:
        print(f"✓ Student Found: {result.first_name} {result.last_name}")
        print(f"  Email: {result.email}")
        print(f"  Student ID: {result.student_id}")
        print(f"Search Time: {search_time:.4f} milliseconds")

    # ========== SECTION 7: Sorting Operations ==========
    print("\n" + "="*70)
    print("SECTION 7: SORTING OPERATIONS WITH TIMING")
    print("="*70)

    print("\n--- Sort Students by Name (Ascending) ---")
    start_time = time.time()
    app.sort_students_by_name()
    sort_time = (time.time() - start_time) * 1000

    print("Sorted List:")
    for i, student in enumerate(app.students[:5], 1):
        print(f"  {i}. {student.first_name} {student.last_name}")
    print(f"Sort Time: {sort_time:.4f} milliseconds")

    print("\n--- Sort Students by Marks (Descending) ---")
    start_time = time.time()
    app.sort_students_by_marks(reverse=True)
    sort_time = (time.time() - start_time) * 1000

    print("Top 5 Students by Average Marks:")
    for i, student in enumerate(app.students[:5], 1):
        avg = student.get_average_marks()
        print(f"  {i}. {student.first_name} {student.last_name}: {avg:.2f}")
    print(f"Sort Time: {sort_time:.4f} milliseconds")

    # ========== SECTION 8: Update Operations ==========
    print("\n" + "="*70)
    print("SECTION 8: UPDATE OPERATIONS")
    print("="*70)

    print("\n--- Updating Student Information ---")
    old_name = added_students[0].first_name
    app.update_student(added_students[0].student_id, first_name="Alicia")
    print(f"✓ Updated: {old_name} -> {added_students[0].first_name}")

    print("\n--- Updating Course Information ---")
    app.update_course("DATA200", description="Advanced data structures with Python implementation")
    print("✓ Course description updated")

    # ========== SECTION 9: Delete Operations ==========
    print("\n" + "="*70)
    print("SECTION 9: DELETE OPERATIONS")
    print("="*70)

    print("\n--- Deleting a Student ---")
    student_to_delete = added_students[-1]
    print(f"Deleting: {student_to_delete.first_name} {student_to_delete.last_name}")
    app.delete_student(student_to_delete.student_id)
    print(f"Remaining students: {len(app.students)}")

    # ========== SECTION 10: User Authentication ==========
    print("\n" + "="*70)
    print("SECTION 10: USER AUTHENTICATION")
    print("="*70)

    print("\n--- Creating User Account ---")
    user = LoginUser("alice.cooper@student.edu", "SecurePass123!", "student")
    print(f"User created: {user.email}")
    print(f"Password encrypted: {user.password_hash[:40]}...")

    print("\n--- Testing Login ---")
    login_result = user.login("SecurePass123!")
    print(f"Login status: {'Success' if login_result else 'Failed'}")

    print("\n--- Testing Password Change ---")
    change_result = user.change_password("SecurePass123!", "NewPassword456!")
    print(f"Password change status: {'Success' if change_result else 'Failed'}")

    # ========== SECTION 11: File Operations ==========
    print("\n" + "="*70)
    print("SECTION 11: CSV FILE OPERATIONS")
    print("="*70)

    print("\n--- Saving Data to CSV Files ---")
    start_time = time.time()
    app.save_all_data()
    save_time = (time.time() - start_time) * 1000
    print(f"Total save time: {save_time:.2f} milliseconds")

    print("\n--- Loading Data from CSV Files ---")
    new_app = CheckMyGradeApp()
    start_time = time.time()
    new_app.load_all_data()
    load_time = (time.time() - start_time) * 1000

    print(f"\nData Loaded:")
    print(f"  Students: {len(new_app.students)}")
    print(f"  Courses: {len(new_app.courses)}")
    print(f"  Professors: {len(new_app.professors)}")
    print(f"Total load time: {load_time:.2f} milliseconds")

    # ========== SECTION 12: Performance Testing ==========
    print("\n" + "="*70)
    print("SECTION 12: LARGE DATASET PERFORMANCE TEST")
    print("="*70)

    print("\n--- Adding 1000 Students ---")
    perf_app = CheckMyGradeApp()
    perf_app.add_course("TEST100", "Test Course", 3)

    start_time = time.time()
    for i in range(1000):
        student = perf_app.add_student(f"TestStudent{i}", f"LastName{i}",
                                       f"test{i}@student.edu")
        marks = random.randint(60, 100)
        perf_app.add_student_grade(student.student_id, "TEST100", marks)
    add_time = (time.time() - start_time) * 1000

    print(f"✓ 1000 students added with grades")
    print(f"Total time: {add_time:.2f} milliseconds")
    print(f"Average time per student: {add_time/1000:.4f} milliseconds")

    print("\n--- Searching in 1000 Records ---")
    start_time = time.time()
    result = perf_app.search_student_by_email("test500@student.edu")
    search_time = (time.time() - start_time) * 1000
    print(f"✓ Search completed in {search_time:.4f} milliseconds")

    print("\n--- Sorting 1000 Records by Name ---")
    start_time = time.time()
    perf_app.sort_students_by_name()
    sort_time = (time.time() - start_time) * 1000
    print(f"✓ Sort completed in {sort_time:.2f} milliseconds")

    print("\n--- Sorting 1000 Records by Marks ---")
    start_time = time.time()
    perf_app.sort_students_by_marks(reverse=True)
    sort_time = (time.time() - start_time) * 1000
    print(f"✓ Sort completed in {sort_time:.2f} milliseconds")

    # ========== FINAL SUMMARY ==========
    print("\n" + "="*70)
    print("DEMONSTRATION COMPLETE")
    print("="*70)
    print("\nAll CSV files have been generated:")
    print("  ✓ students.csv")
    print("  ✓ courses.csv")
    print("  ✓ professors.csv")
    print("  ✓ login.csv")
    print("\nAll functionalities tested successfully!")
    print("="*70)


# Main Execution


if __name__ == "__main__":
    print("Choose an option:")
    print("1. Run Complete Demo")
    print("2. Run Unit Tests")
    print("3. Run Both")

    choice = input("\nEnter your choice (1/2/3): ").strip()

    if choice == "1":
        run_complete_demo()
    elif choice == "2":
        print("\n" + "="*70)
        print(" "*20 + "RUNNING UNIT TESTS")
        print("="*70)
        unittest.main(argv=[''], verbosity=2, exit=False)
    elif choice == "3":
        run_complete_demo()
        print("\n\n")
        print("="*70)
        print(" "*20 + "RUNNING UNIT TESTS")
        print("="*70)
        unittest.main(argv=[''], verbosity=2, exit=False)
    else:
        print("Invalid choice. Running complete demo by default.")
        run_complete_demo()

Choose an option:
1. Run Complete Demo
2. Run Unit Tests
3. Run Both

Enter your choice (1/2/3): 3


               CheckMyGrade Application
               Complete Feature Demonstration

SECTION 1: COURSE MANAGEMENT

--- Adding Courses ---
Course Data Structures and Algorithms added successfully
Course Machine Learning Fundamentals added successfully
Course Database Systems added successfully
Course Big Data Analytics added successfully

Total courses added: 4

SECTION 2: PROFESSOR MANAGEMENT

--- Adding Professors ---
Professor Dr. Michael Johnson added successfully with ID: PROF_20251105011938_H1YC
Professor Dr. Sarah Williams added successfully with ID: PROF_20251105011938_RWLC
Professor Dr. Robert Chen added successfully with ID: PROF_20251105011938_M49Z

Total professors added: 3

--- Professor Course Details ---

Professor ID: PROF_20251105011938_H1YC
Name: Dr. Michael Johnson
Email: michael@mycsu.edu
Rank: Senior Professor
Courses: DATA200, DATA201

Courses Taught:

Course ID: 

test_01_add_student (__main__.TestCheckMyGradeApp.test_01_add_student)
Test adding students to the system ... ok
test_02_delete_student (__main__.TestCheckMyGradeApp.test_02_delete_student)
Test deleting students ... ok
test_03_update_student (__main__.TestCheckMyGradeApp.test_03_update_student)
Test updating student information ... ok
test_04_add_large_student_dataset (__main__.TestCheckMyGradeApp.test_04_add_large_student_dataset)
Test adding 1000+ student records ... ok
test_05_search_performance (__main__.TestCheckMyGradeApp.test_05_search_performance)
Test search performance with large dataset ... ok
test_06_sort_by_name (__main__.TestCheckMyGradeApp.test_06_sort_by_name)
Test sorting students by name ... ok
test_07_sort_by_marks (__main__.TestCheckMyGradeApp.test_07_sort_by_marks)
Test sorting students by marks ... FAIL
test_08_sort_large_dataset (__main__.TestCheckMyGradeApp.test_08_sort_large_dataset)
Test sorting performance with 1000+ records ... 

✓ Sort completed in 622.50 milliseconds

DEMONSTRATION COMPLETE

All CSV files have been generated:
  ✓ students.csv
  ✓ courses.csv
  ✓ professors.csv
  ✓ login.csv

All functionalities tested successfully!



                    RUNNING UNIT TESTS

TEST 1: Adding Students
Student John Doe added successfully with ID: STU_20251105011939_BH7Y
✓ Student added successfully
Execution time: 0.0567 milliseconds

TEST 2: Deleting Students
Student Jane Smith added successfully with ID: STU_20251105011939_GQ5E
Student STU_20251105011939_GQ5E deleted successfully
✓ Student deleted successfully
Execution time: 0.0126 milliseconds

TEST 3: Updating Student Records
Student Bob Johnson added successfully with ID: STU_20251105011939_PSHX
Student STU_20251105011939_PSHX updated successfully
✓ Student updated successfully
Execution time: 0.0145 milliseconds

TEST 4: Adding 1000+ Student Records
Student Student0 Last0 added successfully with ID: STU_20251105011939_WZEQ
Student Student1 Last1 added succe

ok
test_09_course_operations (__main__.TestCheckMyGradeApp.test_09_course_operations)
Test course add/delete/modify operations ... ok


✓ 1000 students sorted by name
Execution time: 220.06 milliseconds
Course Data Structures added successfully
Grade added for student STU_20251105011939_NA3R in course DATA200
Grade added for student STU_20251105011939_UWEJ in course DATA200
Grade added for student STU_20251105011939_9UR7 in course DATA200
Grade added for student STU_20251105011939_N2XZ in course DATA200
Grade added for student STU_20251105011939_B5CL in course DATA200
Grade added for student STU_20251105011939_X1VS in course DATA200
Grade added for student STU_20251105011939_1KCP in course DATA200
Grade added for student STU_20251105011939_OY5S in course DATA200
Grade added for student STU_20251105011939_QTWH in course DATA200
Grade added for student STU_20251105011939_B6CO in course DATA200
Grade added for student STU_20251105011939_40WI in course DATA200
Grade added for student STU_20251105011939_BJKR in course DATA200
Grade added for student STU_20251105011939_ZY8R in course DATA200
Grade added for student STU_20251

test_10_professor_operations (__main__.TestCheckMyGradeApp.test_10_professor_operations)
Test professor add/delete/modify operations ... ok
test_11_file_operations (__main__.TestCheckMyGradeApp.test_11_file_operations)
Test CSV file save and load operations ... ok
test_12_statistics (__main__.TestCheckMyGradeApp.test_12_statistics)
Test course statistics calculation ... ok
test_13_password_encryption (__main__.TestCheckMyGradeApp.test_13_password_encryption)
Test password encryption and verification ... ok

FAIL: test_07_sort_by_marks (__main__.TestCheckMyGradeApp.test_07_sort_by_marks)
Test sorting students by marks
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/tmp/ipython-input-1675080776.py", line 838, in test_07_sort_by_marks
    self.assertEqual(self.app.students[0].first_name, "Bob")
AssertionError: 'Charlie' != 'Bob'
- Charlie
+ Bob


----------------------------------------------------------------------
Ran 13


TEST 10: Professor Operations (Add/Delete/Modify)
Course Data Structures added successfully
Professor Dr. John Smith added successfully with ID: PROF_20251105011939_XSVD
✓ Professor added - Time: 0.0679 ms
Professor PROF_20251105011939_XSVD updated successfully
✓ Professor modified - Time: 0.0138 ms
Professor PROF_20251105011939_XSVD deleted successfully
✓ Professor deleted - Time: 0.0114 ms

TEST 11: CSV File Operations (Save/Load)
Course Data Structures added successfully
Student Test Student added successfully with ID: STU_20251105011939_THVB
Professor Dr. Test added successfully with ID: PROF_20251105011939_F6BM
Grade added for student STU_20251105011939_THVB in course DATA200
Students saved to students.csv
Courses saved to courses.csv
Professors saved to professors.csv
No users to save

All data saved successfully!
✓ All data saved - Time: 0.6385 ms
Students loaded from students.csv
Courses loaded from courses.csv
Professors loaded from professors.csv
File login.csv not found

Al